<a href="https://colab.research.google.com/github/NyusenKyusai/18018535_DataAnalytics/blob/main/TrainingStrongLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I will creating a model with the dataset that is temperature, dew point, and has the strong outliers removed.

In [1]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df1 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/linearTDStrong.csv', index_col=0, )
df2 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/linearTDSnowStrong.csv', index_col=0, )
df3 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/linearTDMaxMinStrong.csv', index_col=0, )
df4 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/linearTDMaxMinSnowStrong.csv', index_col=0, )

In [3]:
# make sure we have our data by printing it out
print(df1[:6])

   temp  dewp  NUM_CRIMES
2  25.7  16.0         674
3  31.5  27.0         648
4  29.0  26.1         514
5   2.2  -7.4         523
6   5.3  -0.9         503
7   3.1  -7.6         474


In [146]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df4.iloc[np.random.permutation(len(df4))]

# df1 = 2
# df1 = 3
# df1 = 4
# df1 = 5
# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:5]

# print out the first 6 rows of predictors.
print(predictors[:6])

      temp  dewp   max   min  sndp
398   34.5  31.6  45.0  30.0   0.0
435   49.0  40.9  63.0  45.0   0.0
22    29.9  22.7  33.1  28.0   0.0
535   74.3  51.2  87.1  57.9   0.0
1540  41.7  33.2  48.9  30.2   0.0
1824  51.9  49.0  57.2  44.1   0.0


In [147]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,temp,dewp,max,min,sndp,NUM_CRIMES
398,34.5,31.6,45.0,30.0,0.0,614
435,49.0,40.9,63.0,45.0,0.0,716
22,29.9,22.7,33.1,28.0,0.0,752
535,74.3,51.2,87.1,57.9,0.0,763
1540,41.7,33.2,48.9,30.2,0.0,693


In [148]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

398     614
435     716
22      752
535     763
1540    693
1824    689
Name: NUM_CRIMES, dtype: int64


In [149]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_TRIPS = 1.0

In [150]:
# Split our data into a training set i.e. 75% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_CRIMES'])*0.75)
# Split our data into a test set i.e. 20% of the length of the shuffle array
testsize = int(len(shuffle['NUM_CRIMES'])*0.2)
# The final test set size is 100% - 75% - 20% = 5% of the length of the shuffle array.
finaltestsize = len(shuffle['NUM_CRIMES']) - trainsize - testsize

# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

In [151]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
# linearTDStrong
# linearTDSnowStrong
# linearTDMaxMinStrong
# linearTDMaxMinSnowStrong
shutil.rmtree('/tmp/linearTDMaxMinSnowStrong5', ignore_errors=True)

# This is the core of our linear regressor

# You can see that we save the model, use the the Adam optimization algorithm, which is an extension 
# to stochastic gradient descent that has recently seen broader adoption for deep learning applications 
# in computer vision and natural language processing and infer real valued columns from input which interprets 
# all inputs as dense, fixed-length float values.

# See the link for more information
# https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/contrib/learn/LinearRegressor
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linearTDMaxMinSnowStrong5', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_TRIPS, steps=10000)

# Next, we can check our predictions based on our predictors.
# It goes from trainsize to testsize + trainsize to keep the last 5% out of the model
# We can test with the last 5% later
preds = estimator.predict(x=predictors[trainsize:testsize + trainsize].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_NUM_TRIPS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:testsize + trainsize].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Number of Crimes Values.
avg = np.mean(shuffle['NUM_CRIMES'][:trainsize])

# Calculate the RMSE using Number of Crimes Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_CRIMES'][trainsize:testsize + trainsize] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

1.15.2
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f07ac7390>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linearTDMaxMinSnowStrong5', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running 

Temp Dewp  
Model 1 got a RMSE of 56.4476 | Using the average of 730.0410 it got an RMSE of 78.1526  
Model 2 got a RMSE of 55.5389 | Using the average of 729.6186 it got an RMSE of 82.3526  
Model 3 got a RMSE of 56.4135 | Using the average of 727.0608 it got an RMSE of 79.9334  
Model 4 got a RMSE of 63.9391 | Using the average of 729.1464 it got an RMSE of 86.8047  
Model 5 got an RMSE of 58.6262 | Using the average of 728.5022 it got an RMSE of 88.3506   


Temp Dewp Sndp  
Model 1 got a RMSE of 58.1260 | Using the average of 729.2819 it got an RMSE of 79.8057  
Model 2 got a RMSE of 57.2818 | Using the average of 728.4758 it got an RMSE of 82.2580  
Model 3 got a RMSE of 53.2411 | Using the average of 727.3089 it got an RMSE of 81.4985  
Model 4 got a RMSE of 62.9393 | Using the average of 729.6435 it got an RMSE of 86.3034  
Model 5 got an RMSE of 56.6542 | Using the average of 730.5483 it got an RMSE of 84.1646   
  

Temp Dewp Max Min  
Model 1 got a RMSE of 55.5577 | Using the average of 730.0608 it got an RMSE of 83.7190  
Model 2 got a RMSE of 56.8788 | Using the average of 730.3353 it got an RMSE of 83.2992  
Model 3 got a RMSE of 55.6406 | Using the average of 730.0469 it got an RMSE of 78.3270  
Model 4 got a RMSE of 60.3791 | Using the average of 727.8851 it got an RMSE of 85.0037  
Model 5 got an RMSE of 54.5984 | Using the average of 728.8038 it got an RMSE of 83.8934   
  

Temp Dewp Max Min  
Model 1 got a RMSE of 56.8273 | Using the average of 729.1808 it got an RMSE of 75.8665  
Model 2 got a RMSE of 60.8201 | Using the average of 730.7328 it got an RMSE of 88.3429  
Model 3 got a RMSE of 58.5375 | Using the average of 729.8660 it got an RMSE of 85.3575  
Model 4 got a RMSE of 61.5252 | Using the average of 729.8521 it got an RMSE of 85.1009  
Model 5 got an RMSE of 59.9994 | Using the average of 727.3968 it got an RMSE of 76.6655   
  

In [152]:
# Creating a variable for the last 5% of the data that is easy to handle
input = predictors[testsize + trainsize:]
					
# Boilerplate for creating the estimator
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linearTDMaxMinSnowStrong5', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

# Predicting the values 
preds = estimator.predict(x=input.values)

# Creating a list of predictions 
predslist = preds['scores']*SCALE_NUM_TRIPS

# Creating the RMSE by comparing the targets to the list of predictions
rmse = np.sqrt(np.mean((targets[testsize + trainsize:].values - predslist)**2))

# Calculate the mean of the Number of Crimes Values.
avg = np.mean(shuffle['NUM_CRIMES'][testsize + trainsize:])

corr_matrix = np.corrcoef(targets[testsize + trainsize:].values, predslist)
corr = corr_matrix[0,1]
R_sq = corr**2

print()
# Printing the rmse and average crimes
print('LinearRegression has RMSE of {0}'.format(rmse));
print('Average is {0}'.format(avg))
print('The coefficient of determination is {0}'.format(R_sq))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4f0b2eb790>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linearTDMaxMinSnowStrong5', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linearTDMaxMinSnowStrong5/model.ckpt-10000
INF

Temp Dewp  
Model 1 got a RMSE of 56.4245 and a r^2 of 0.6091  
Model 2 got a RMSE of 56.4859 and a r^2 of 0.5122  
Model 3 got a RMSE of 71.7048 and a r^2 of 0.4648  
Model 4 got a RMSE of 58.4935 and a r^2 of 0.4869  
Model 5 got an RMSE of 59.4891 and a r^2 of 0.5347  

Temp Dewp Sndp  
Model 1 got a RMSE of 52.2847 and a r^2 of 0.6171  
Model 2 got a RMSE of 60.7817 and a r^2 of 0.5003  
Model 3 got a RMSE of 64.2587 and a r^2 of 0.5034  
Model 4 got a RMSE of 53.4029 and a r^2 of 0.5930  
Model 5 got an RMSE of 56.9881 and a r^2 of 0.4893  

Temp Dewp Max Min  
Model 1 got a RMSE of 58.2913 and a r^2 of 0.5469  
Model 2 got a RMSE of 58.6183 and a r^2 of 0.5374  
Model 3 got a RMSE of 51.5437 and a r^2 of 0.5308  
Model 4 got a RMSE of 66.7020 and a r^2 of 0.4257  
Model 5 got an RMSE of 67.3570 and a r^2 of 0.4535  


Temp Dewp Max Min Sndp  
Model 1 got a RMSE of 55.6091 and a r^2 of 0.5127  
Model 2 got a RMSE of 54.3686 and a r^2 of 0.6086  
Model 3 got a RMSE of 63.3079 and a r^2 of 0.4911  
Model 4 got a RMSE of 61.4333 and a r^2 of 0.4896  
Model 5 got an RMSE of 58.8918 and a r^2 of 0.5092  